# Planning-Lab Lesson 2: Informed Search Strategies

In the second session we will work on informed search

### Maze Environments
The environments used is **SmallMaze** (visible in the figure).

<img src="images/maze.png" width="300">

The agent starts in cell $(0, 2)$ and has to reach the treasure in $(4, 3)$.

### Priority Queue

You will need a queue ordered by priority as a queue or **PriorityQueue**. The difference between the other versions of the queue is that in **PriorityQueue**, nodes are removed from the data structure based on the current lowest value. In particular, **Node** has two useful parameters (other than those used in the previous session):

- pathcost - the path cost from the root node to the current one (defaults to 0)
- value - the value of a node. Used by PriorityQueue to order its content (defaults to 0)

### Here is an example of usage:

In [3]:
import os
import sys
module_path = os.path.abspath(os.path.join('../tools'))
if module_path not in sys.path:
    sys.path.append(module_path)

from utils.ai_lab_functions import *

# Create 3 nodes for state ids 1 2 3
node_1 = Node(1) # No parent, pathcost=0, value=0
node_2 = Node(2, node_1, node_1.pathcost + 1, 10) # Child of node_1, pathcost=1, value=10
node_3 = Node(3, node_1, 100, 5)  # Child of node_1, pathcost=100, value=5

p_queue = PriorityQueue()
for n in (node_1, node_2, node_3):
    p_queue.add(n)
    print("Added: {}".format(n.state))

while not p_queue.is_empty():
    print("Removed: {}".format(p_queue.remove().state))

ModuleNotFoundError: No module named 'matplotlib'

Notice the order in which nodes are removed from the queue.

## Uniform-Cost Search (UCS)
Before moving to informed search, it is useful to see another uninformed search algorithm: the Uniform-Cost Search (UCS). You can see the implementation in the *graph search* version in the following. The cost of performing an action is supposed to be 1 (also in the assignments).

In [ ]:
def present_with_higher_cost(queue, node):
    if node.state in queue:
        if queue[node.state].value > node.value: 
            return True
    return False

In [ ]:
import gym
import envs

def ucs(environment):
    """
    Uniform-cost search
    
    Args:
        environment: OpenAI Gym environment
        
    Returns:
        path: solution as a path
    """

    # Used to cross a weighted tree or graph. Is useful when it comes to study each edge with different weight
    # Goal of UCS -> find a path to the goal node that has the lowest cumulative cost
    # Expand all the nodes starting from the root or start state (it depends if we study a tree or a graph)
    # OPTIMAL -> AT EVERY ITERATION THE PATH WITH THE LEAST COST IS CHOSEN
    # DISATVANTAGES -> does not care about the number of state but it cares only about the path cost

    # siccome e' basato sul costo di ogni arco/nodo devo inizializzare una priority queue. INIT -> natural ordering
    queue = PriorityQueue()
    # aggiungo il nodo alla queue con il suo peso
    queue.add(Node(environment.startstate))
    
    # mantengo la memoria dei nodi esplorati
    explored = set()
    time_cost = 1
    space_cost = 1
    
    # inizio esplorazione
    while True:
        # se la queue e' vuota significa che non sono riuscito a trovare una soluzione
        if queue.is_empty(): 
            return None
        
        # Retrieve node from the queue
        node = queue.remove()
        # controllo se ho trovato il goal state
        if node.state == environment.goalstate: 
            # se lo trovo restituisco la soluzione
            return build_path(node), time_cost, space_cost
        
        # aggiungo il nodo ai nodi esplorati
        explored.add(node.state)
        
        # Look around
        for action in range(environment.action_space.n):
            
            # Child node where value and pathcost are both the pathcost of parent + 1
            child = Node(environment.sample(node.state, action), node, node.pathcost + 1, node.pathcost + 1)  
            time_cost += 1
            
            # se il nodo figlio queue non e' nella queue e non e' in explred
            if child.state not in queue and child.state not in explored:
                # aggiungere alla queue
                queue.add(child)
            
            # sostituire il figlio con il costo piu' alto in modo da rimpiazzarlo
            elif present_with_higher_cost(queue, child):
                # sostituisci il figlio in questo caso
                queue.replace(child)
                
        space_cost = max(space_cost, len(queue) + len(explored))


#### Let's see the results:

In [ ]:
# Create and render the environment
env = gym.make("SmallMaze-v0")
env.render()
solution, time, memory = ucs(env)

CheckResult_UCS(solution, time, memory, env)

[['C' 'C' 'S' 'C']
 ['C' 'C' 'W' 'C']
 ['C' 'C' 'C' 'C']
 ['C' 'W' 'W' 'W']
 ['C' 'C' 'C' 'G']]

##########################################
#####  UNIFORM GRAPH SEARCH PROBLEM  #####
##########################################
Solution: [(0, 1), (0, 0), (1, 0), (2, 0), (3, 0), (4, 0), (4, 1), (4, 2), (4, 3)]
N° of nodes explored: 61
Max n° of nodes in memory: 16


## Distance Heuristics

Informed search requires a distance heuristic to estimate the distance between a state and the goal. You already have at your disposal these functions:

- *l1_norm(p1, p2)* - Computes the L1 norm (also known as the manhattan distance) between two points specified as tuples of coordinates.
- *l2_norm(p1, p2)* - Computes the L2 norm between two points specified as tuples of coordinates.
- *chebyshev(p1, p2)* - Computes the Chebyshev distance between two points specified as tuples of coordinates. Similar to the L1 norm but diagonal moves are also considered.


**Examples:**

In [ ]:
p1 = (0, 2)
p2 = (4, 0)
print("L1 norm heuristic value: {}".format(Heu.l1_norm(p1, p2)))
print("L2 norm heuristic value: {}".format(Heu.l2_norm(p1, p2)))
print("Chebyshev heuristic value: {}".format(Heu.chebyshev(p1, p2)))

L1 norm heuristic value: 6
L2 norm heuristic value: 4.47213595499958
Chebyshev heuristic value: 4


# Assignment 1: Greedy Best-First Search

The first assignment is to implement the Greedy-best-first search algorithm on **SmallMaze**. In particular, you have to implement both *greedy_tree_search* and *greedy_graph_search* versions that will be called by the generic *greedy*. Use the L1 norm as a heuristic function first, then try the others to see the differences.

The results returned by greedy must be in the following form (path, time_cost, space_cost), more in detail:

- **path** - a tuple of state identifiers forming a path from the start state to the goal state. None if no solution is found.
- **time_cost** - the number of nodes checked during the exploration.
- **space_cost** - the maximum number of nodes in memory simultaneously.


### Functions to implement:
- *greedy_tree_search(environment)*
- *greedy_graph_search(environment)*

In [ ]:
def greedy_tree_search(environment, timeout=10000):
    """
    Greedy-best-first Tree search
    
    Args:
        problem: OpenAI Gym environment
        
    Returns:
        (path, time_cost, space_cost): solution as a path and stats.
    """

    # Informed search algorithm
    # we need a value and an heu5ristic value in order to make the algorithm work
    # we have an open list and a close list -> the close list is the solution and in order to put nodes in that we have to
    # search for the least heuristic value [our case is l1 norm heuristic, l2 norm heuristic and chebyshev heuristic]#

    # we need to find the goal state 
    goalpos = environment.state_to_pos(environment.goalstate)
    # initialisation of the priority queue
    queue = PriorityQueue()
    
    # root node
    node = Node(environment.startstate)
    # adding the node to the queue
    queue.add(node)
    
    # initialise time_cost and space_cost
    time_cost = 1
    space_cost = 1
    
    max_space_cost = 1
    
    while True:
        # if that condition is true return a time-out
        if time_cost >= timeout: 
            return ("time-out", time_cost, space_cost) # timeout check
        # if queue is empty return a failure
        elif queue.is_empty():
            return "failure", time_cost, space_cost
        
        # remove the first node from the queue (highest priority)
        node = queue.remove()
        # decrease the space cost
        space_cost -= 1
        
        # if the node removed is actually the node state
        if node.state == environment.goalstate:
            # return the solution to this problem
            return build_path(node), time_cost, max_space_cost
        
        # search for nearest children
        for action in range(environment.action_space.n):
            # retrieve the child state
            child_state = environment.sample(node.state, action)
            
            # create the son
            p1 = environment.state_to_pos(child_state)
            # give the node one of the three heuristics
            node_value = Heu.l1_norm(p1, goalpos)
            
            # child is formed with the weight given by the heuristic
            child = Node(child_state, node, node.pathcost + 1, node_value)
            
            # increase the time_cost
            time_cost += 1
            
            # adding the child to the queue
            queue.add(child)
            # increasing the space cost
            space_cost += 1
            
            # calculate the max space cost
            max_space_cost = max(space_cost, max_space_cost)
            
            # replace the child just in case
            if present_with_higher_cost(queue, child):
                queue.replace(child)
        
    return None, time_cost, max_space_cost

In [ ]:
def greedy_graph_search(environment):
    """
    Greedy-best-first Graph search
    
    Args:
        problem: OpenAI Gym environment
        
    Returns:
        (path, time_cost, space_cost): solution as a path and stats.
    """
    

    # algorithm pretty similar to tree-search

    goalpos = environment.state_to_pos(environment.goalstate)
    queue = PriorityQueue()
    
    node = Node(environment.startstate)
    queue.add(node)
    
    explored = set()
    time_cost = 1
    space_cost = 1
    
    max_space_cost = 1
    
    while True:
        if queue.is_empty():
            return "failure", time_cost, space_cost
        
        node = queue.remove()
        explored.add(node.state)
        
        if node.state == environment.goalstate:
            return build_path(node), time_cost, max_space_cost
        
        for action in range(environment.action_space.n):
            child_state = environment.sample(node.state, action)
            
            p1 = environment.state_to_pos(child_state)
            node_value = Heu.l1_norm(p1, goalpos)
            
            child = Node(child_state, node, node.pathcost + 1, node_value)
            
            time_cost += 1
            
            if child.state not in queue and child.state not in explored:
                queue.add(child)
                space_cost += 1
                max_space_cost = max(space_cost, max_space_cost)
            elif present_with_higher_cost(queue, child):
                queue.replace(child)
        
    return None, time_cost, max_space_cost

**The following function calls your implementations of greedy_tree_search and greedy_graph_search:**

In [ ]:
def greedy(environment, search_type):
    """
    Greedy-best-first search
    
    Args:
        problem: OpenAI Gym environment
        search_type: type of search - greedy_tree_search or greedy_graph_search (function pointer)
        
    Returns:
        (path, time_cost, space_cost): solution as a path and stats.
    """
    
    path, time_cost, space_cost = search_type(environment)
    return path, time_cost, space_cost

**The following code calls your *tree search* and *graph search* version of Greedy-best-first search and checks the results:**

In [ ]:
envname = "SmallMaze-v0"
environment = gym.make(envname)

solution_ts, time_ts, memory_ts = greedy(environment, greedy_tree_search)
solution_gs, time_gs, memory_gs = greedy(environment, greedy_graph_search)
heuristic = str(input('Which heuristic did you use? (l1_norm, l2_norm, chebyshev)\n'))

results = CheckResult_L2A1([solution_ts, time_ts, memory_ts], [solution_gs, time_gs, memory_gs], heuristic, env)
results.check_sol_ts()
results.check_sol_gs()

########################################################
#######  GREEDY BEST FIRST TREE SEARCH PROBLEM  ########
########################################################
Your solution: time-out
N° of nodes explored: 10001
Max n° of nodes in memory: 7501

===> Your solution is correct!

########################################################
#######  GREEDY BEST FIRST GRAPH SEARCH PROBLEM  #######
########################################################
Your solution: [(0, 3), (1, 3), (2, 3), (2, 2), (2, 1), (2, 0), (3, 0), (4, 0), (4, 1), (4, 2), (4, 3)]
N° of nodes explored: 45
Max n° of nodes in memory: 15

===> Your solution is correct!



# Assignment 2: A* Search
The second assignment is to implement the A search algorithm on SmallMaze. In particular, you have to implement both astar_tree_search and astar_graph_search versions that the generic astar will call. Use the L1 norm* as a heuristic function first, then try the others to see the differences.

The results returned by astar must be in the following form (path, time_cost, space_cost), more in detail:

- **path** - a tuple of state identifiers forming a path from the start state to the goal state. None if no solution is found.
- **time_cost** - the number of nodes checked during the exploration.
- **space_cost** - the maximum number of nodes in memory simultaneously.

Functions to implement:
- *astar_tree_search(environment)*
- *astar_graph_search(environment)*

In [ ]:
def astar_tree_search(environment):
    """
    A* Tree search
    
    Args:
        problem: OpenAI Gym environment
        
    Returns:
        (path, time_cost, space_cost): solution as a path and stats.
    """

    # https://www.youtube.com/watch?v=PzEWHH2v3TE&ab_channel=Education4u
    # video che spiega A* graph search che è comunque molto simile al tree search quindi la spiegazione serve per capire entrambi#

    # mi salvo la posizione del nodo goal
    goalpos = environment.state_to_pos(environment.goalstate)
    queue = PriorityQueue()
    
    node = Node(environment.startstate)
    queue.add(node)
    
    time_cost = 1
    space_cost = 1
    
    max_space_cost = 1
    
    while True:
        # e la queue è vuota significa che non ho trovato la soluzione e quindi restituisco un fallimento
        if queue.is_empty():
            return "failure", time_cost, space_cost
        
        # rimuovo il nodo dalla queue e diminuisco lo spazio occupato nella totalità
        node = queue.remove()
        space_cost -= 1
        
        # se il nodo è effettivamente il goal state restituisco la soluzione
        if node.state == environment.goalstate:
            return build_path(node), time_cost, max_space_cost
        
        # controllo i nodi figlio
        for action in range(environment.action_space.n):
            child_state = environment.sample(node.state, action)
            
            # conferisco un'euristica ai nodi figli
            p1 = environment.state_to_pos(child_state)
            node_value = Heu.l1_norm(p1, goalpos) + (node.pathcost + 1)
            
            # "creo" il nodo figlio per inserirlo nella queue
            child = Node(child_state, node, node.pathcost + 1, node_value)
            
            time_cost += 1
            
            # non tengo conto che il figlio possa già essere stato esplorato o no
            queue.add(child)
            space_cost += 1
            
            max_space_cost = max(space_cost, max_space_cost)
            
            # rimpiazzo il figlio nella queue se è già presente nella queue ma con un costo minore
            if present_with_higher_cost(queue, child):
                queue.replace(child)
    
    return None, time_cost, space_cost

In [ ]:
def astar_graph_search(environment):
    """
    A* Graph Search
    
    Args:
        problem: OpenAI Gym environment
        
    Returns:
        (path, time_cost, space_cost): solution as a path and stats.
    """

    # For each edge you have to consider the weight from one vertice to another in order to calculate it
    # In order to make the algorithm work we need 2 lists basically: states and their heuristic values
    # calulating the values of f(n) = g(n) + h(n) we consider the path with less weight instead of the sortest one#

    # trovo il goal
    goalpos = environment.state_to_pos(environment.goalstate)
    # utilizzo la priority queue perchè ho un'euristica
    queue = PriorityQueue()
    
    # trovo lo start state
    node = Node(environment.startstate)
    queue.add(node)
    
    # tengo conto dei nodi esplorati
    explored = set()
    time_cost = 1
    space_cost = 1

    max_space_cost = 1
    
    while True:
        # se la queue è vuota -> non ho trovato la soluzione al problema
        if queue.is_empty():
            return "failure", time_cost, space_cost
        
        # rimuovo il nodo dalla queue degli stati da esplorare e lo metto negli esplorati
        node = queue.remove()
        explored.add(node.state)
        
        # se il nodo che ho rimosso è effettivamente il goal state restituisco la soluzione
        if node.state == environment.goalstate:
            return build_path(node), time_cost, max_space_cost
        
        # esploro i nodi direttamente collegati a quello che ho appena "studiato"
        for action in range(environment.action_space.n):
            # controllo il nodo "figlio"
            child_state = environment.sample(node.state, action)
            
            # gli conferisco un'euristica (può essere l1_norm, l2_norm o chebyshev)
            p1 = environment.state_to_pos(child_state)
            node_value = Heu.l1_norm(p1, goalpos) + (node.pathcost + 1)
            
            child = Node(child_state, node, node.pathcost + 1, node_value)
            
            time_cost += 1
            
            # se il nodo nuovo non è nella queue e non è in explored aggiungo il figlio alla queue e aggiorno max_space_cost
            if child.state not in queue and child.state not in explored:
                queue.add(child)
                space_cost += 1
                max_space_cost = max(space_cost, max_space_cost)
            # se ho già un figlio ma ha un "costo" più alto lo rimpiazzo nella queue
            elif present_with_higher_cost(queue, child):
                queue.replace(child)
    
    return None, time_cost, space_cost

**The following function calls your implementations of astar_tree_search and astar_graph_search:**

In [ ]:
def astar(environment, search_type):
    """
    A* search
    
    Args:
        environment: OpenAI Gym environment
        search_type: type of search - astar_tree_search or astar_graph_search (function pointer)
        
    Returns:
        (path, time_cost, space_cost): solution as a path and stats.
    """
    
    path, time_cost, space_cost = search_type(environment)
    return path, time_cost, space_cost

**The following code calls your *tree search* and *graph search* version of A* search and checks the results:**

In [ ]:
envname = "SmallMaze-v0"
environment = gym.make(envname)

solution_ts, time_ts, memory_ts = astar(environment, astar_tree_search)
solution_gs, time_gs, memory_gs = astar(environment, astar_graph_search)
heuristic = str(input('Which heuristic did you use? (l1_norm, l2_norm, chebyshev)\n'))

results = CheckResult_L2A2([solution_ts, time_ts, memory_ts], [solution_gs, time_gs, memory_gs], heuristic, env)
results.check_sol_ts()
results.check_sol_gs()

#########################################
#######  A* TREE SEARCH PROBLEM  ########
#########################################
Your solution: [(0, 1), (1, 1), (2, 1), (2, 0), (3, 0), (4, 0), (4, 1), (4, 2), (4, 3)]
N° of nodes explored: 8361
Max n° of nodes in memory: 6271

===> Your solution is correct!

##########################################
#######  A* GRAPH SEARCH PROBLEM  ########
##########################################
Your solution: [(0, 1), (1, 1), (2, 1), (2, 0), (3, 0), (4, 0), (4, 1), (4, 2), (4, 3)]
N° of nodes explored: 61
Max n° of nodes in memory: 16

===> Your solution is correct!



### Discussion
Now that you have correctly implemented both Greedy-best-first and A* what can you say about the solutions they compute? Are there significant differences in the stats? Try to play with other heuristics as well and see if your results change.